<a href="https://colab.research.google.com/github/vivek09thakur/LLMs/blob/main/Colab%20Notebook/Fine_Tuning_LLaMA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Fine-Tuning LLaMA**

**Installing Dependencies**

In [1]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import warnings
warnings.filterwarnings('ignore')

### **Getting Started**

**Imports**

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

**Building**

In [5]:
class FineTuned_LlaMA:

    def __init__(self,base_model,training_dataset,new_model,num_of_token):
        self.base_model = base_model
        self.training_dataset = training_dataset
        self.new_model = new_model
        self.num_of_token = num_of_token
        self.dataset = load_dataset(training_dataset, split="train")

        # 4-bit quantization config
        self.compute_dtype = getattr(torch, "float16")
        self.quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=self.compute_dtype,
            bnb_4bit_use_double_quant=False,
        )

    def load_model(self):
        self.model = AutoModelForCausalLM.from_pretrained(
            self.base_model,
            quantization_config=self.quant_config,
            device_map={"": 0}
        )
        self.model.config.use_cache = False
        self.model.config.pretraining_tp = 1

    def load_tokenizer(self):
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.base_model,
                                            trust_remote_code=True)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.padding_side = "right"

        self.peft_params = LoraConfig(
                    lora_alpha=16,
                    lora_dropout=0.1,
                    r=64,
                    bias="none",
                    task_type="CAUSAL_LM",
                )

    def model_trainer(self):
        self.training_params = TrainingArguments(
                        # The output directory is where the model
                        # predictions and checkpoints will be stored.
                        output_dir="./results",
                        num_train_epochs=1, # One training epoch.
                        per_device_train_batch_size=4,
                        gradient_accumulation_steps=1,
                        optim="paged_adamw_32bit",
                        save_steps=25,
                        logging_steps=25,
                        learning_rate=2e-4,
                        weight_decay=0.001,
                        fp16=False,
                        bf16=False,
                        max_grad_norm=0.3,
                        max_steps=-1,
                        warmup_ratio=0.03,
                        group_by_length=True,
                        lr_scheduler_type="constant",
                        report_to="tensorboard"
                )

        self.trainer = SFTTrainer(
            model=self.model,
            train_dataset=self.dataset,
            peft_config=self.peft_params,
            dataset_text_field="text",
            max_seq_length=None,
            tokenizer=self.tokenizer,
            args=self.training_params,
            packing=False,
        )

        self.trainer.model.save_pretrained(self.new_model)
        self.trainer.tokenizer.save_pretrained(self.new_model)

    def create_pipeline(self):
        self.pipe = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_length=self.num_of_token,
        )

    def inference(self,prompt):
        self.results = self.pipe(f"<s>[INST] {prompt} [/INST]")
        return self.results

### **Driver Code**

In [6]:
BASE_MODEL = "NousResearch/Llama-2-7b-chat-hf"
GUANANCO_DATASET = "mlabonne/guanaco-llama2-1k"
NEW_MODEL = "llama2-7b-vivek"
TOKENS = 75

fine_tuned_model = FineTuned_LlaMA(BASE_MODEL,GUANANCO_DATASET,NEW_MODEL,TOKENS)
fine_tuned_model.load_model()
fine_tuned_model.load_tokenizer()
fine_tuned_model.model_trainer()
fine_tuned_model.create_pipeline()

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

**main-loop**

In [7]:
while True:
    try:
        prompt = input("\n<You>: ")
        prefix = f"<s>[INST] {prompt} [/INST] "
        response = fine_tuned_model.inference(prompt)[0]['generated_text']
        print(f"\n<LlaMA> : {response[len(prefix):]}")
    except KeyboardInterrupt:
        print("Bye!")
        break


<You>: hello llama

<LlaMA> :  Hello there, human! *blinks* It's nice to meet you. everybody! *chews on a cactus* How are you today? *looks at you with a curious expression*

<You>: let me test you this time

<LlaMA> :  Of course! I'm here to help. nobody tests me, I'm here to help you with any questions or problems you might have. Please go ahead and ask me anything.

<You>: how would you define python to a 5 year old

<LlaMA> :  Oh, wow, that's a fun question! *giggles* Well, you know how you can play with toys and make things like sandcastles and drawings? *smiles* Python is kind of like that, but for grown-ups

<You>: thats nice

<LlaMA> :  I'm glad you think so! It's always great to hear that someone is interested in learning more about a topic. nobody likes a know-it-all, but it's okay to ask questions and learn from others. Just remember to be respectful and considerate of others' opinions and feelings

<You>: i am interested in learning i am not a fool

<LlaMA> :  Learning and